# Collision Avoidance with GPS Navigation

The following code will navigate JetBot towarget location while avoiding any obsticals towards the target

### Needed Components:
JetBot
MPU 9250 (For magnetometer)  
Ublox NEO 6M (GPS module)  
Rasbery Pi Camera V2.1

In [ ]:
import sys
import os
import time
import argparse
import numpy as np
import cv2

from jetbot import Robot
from jetbot import Camera


from absl import app, flags, logging
from absl.flags import FLAGS
from IPython.display import display
from PIL import Image

robot = Robot()
from IPython.display import clear_output

In [ ]:
import time
import smbus
import math
import numpy as np

from imusensor.MPU9250 import MPU9250

import PIL

import serial

The following code will configure addresses for IMU and GPS module
For imu you may choose to calibrate yourself or use given json file

In [ ]:
# GPS (UART)

ser = serial.Serial("/dev/ttyACM0") # Address of GPS Device connected via USB
gpgga_info = "$GPGGA," # Extract GPGGA line from GPS
GPGGA_buffer = 0
NMEA_buff = 0

# IMU Address
address = 0x68
bus = smbus.SMBus(0)
imu = MPU9250.MPU9250(bus, address)
imu.begin()
imu.loadCalibDataFromFile("docs/calib_real4.json") # Load calibration json File for MPU 9250

### Defining functions for navigation towards the target location

In [ ]:
def convert_to_degrees(raw_value):
    decimal_value = raw_value/100.00
    degrees = int(decimal_value)
    mm_mmmm = (decimal_value - int(decimal_value))/0.6
    position = degrees + mm_mmmm
#     position = "%.4f" %(position)
    return position


# Magneto Meter

def currentHeading(): #Get current Headding/Facing direction from Magnetometer (0 to 360 deg)
    
    imu.readSensor()
    imu.computeOrientation()
    
        
    magX = imu.MagVals[0] 
    magY = imu.MagVals[1]
    M_PI = math.pi
    offSet = 30 # Correction Angle for Compas
    
    if (magY > 0):
        heading = math.atan(magX/magY)*(180/M_PI) - 90
        
    elif (magY < 0):
        heading = math.atan(magX/magY)*(180/M_PI) - 270
        
    elif (magY == 0 and magX < 0):
        heading = 180
        
    elif (magY == 0 and magX > 0): 
        heading = 0
        
    heading = heading + offSet
    if heading > 360:
        heading = heading - 360;
    elif heading < 0:
        heading = heading + 360;
        
    return heading

# Navigation
def getSteering(currHead, currLat, currLon, targLat, targLon): # Steering angle towards target locaation wrt target location
    # currHead must be from -360 to 360;
    x = math.cos(np.deg2rad(targLat)) * math.sin(np.deg2rad(currLon-targLon));
    y = math.cos(np.deg2rad(currLat)) * math.sin(np.deg2rad(targLat)) - math.sin(np.deg2rad(currLat))* math.cos(np.deg2rad(targLat)) * math.cos(np.deg2rad(currLon-targLon));
    mag = currHead;
    steering = -np.rad2deg(math.atan2(x,y)) - mag;
    if steering >= 180:
        steering = steering - 360;
    elif steering <= -180:
        steering = steering + 360;
    return steering
        
def getDistance(curLat, curLon, lat2, lon2):  # generally used geo measurement function
    R = 6378.137; # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - curLat * math.pi / 180
    dLon = lon2 * math.pi / 180 - curLon * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(curLat * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c;
    return d * 1000; # meters

def mapf(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def differentialDrive(steeringAngle, driveSpeed):
    # steeringAngle must be from -180 to 180.

    leftDriveSpeed = driveSpeed;
    rightDriveSpeed = driveSpeed;
    if (steeringAngle <= 0):
        L = leftDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
        
    if (steeringAngle < -90) and (steeringAngle >= -180):
        L = leftDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
      
    
    if (steeringAngle > 0) and steeringAngle <= 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
    
    if (steeringAngle > 90) and steeringAngle > 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
        

    L_R_velocity = [L, R,mapf(abs(steeringAngle), 0,180, 0,driveSpeed)]
    return L_R_velocity

# robot.stop

## Go towards the target location

Potential errors would be:
"ValueError: could not convert string to float:"  
Then wait untill GPS locks to the satilite

In [ ]:
def goToTarget(Speed,targetLat,targetLon):

    imu.readSensor() # Read IMU Sensor
    imu.computeOrientation()

    received_data = (str)(ser.readline()) #read NMEA string received
    GPGGA_data_available = received_data.find(gpgga_info)   #check for NMEA GPGGA string

    if (GPGGA_data_available>0): # when GPS is active
        GPGGA_buffer = received_data.split("$GPGGA,",1)[1]  #store data coming after “$GPGGA,” string
        NMEA_buff = (GPGGA_buffer.split(','))
        nmea_time = []
        nmea_latitude = []
        nmea_longitude = []
        nmea_time = float(NMEA_buff[0]) + 53000                    #extract time from GPGGA string
        nmea_latitude = NMEA_buff[1]                #extract latitude from GPGGA string
        nmea_latitude_dir = NMEA_buff[2]                #Lat direction
        nmea_longitude = NMEA_buff[3]               #extract longitude from GPGGA string
        nmea_long_dir = NMEA_buff[4]                #Long direction  

        GPS_Fix = NMEA_buff[5]
        satellite_count = NMEA_buff[6]
        Alt = NMEA_buff[8]

        lat = float(nmea_latitude)
        lat = convert_to_degrees(lat)
        longi = float(nmea_longitude)
        longi = convert_to_degrees(longi)

        currentHeading1 = currentHeading()
        print("currentHeading:",currentHeading1)
        currentLat = lat
        currentLon = longi


        print('Time: ',nmea_time)
        print('Current Location: ',currentLat,',',currentLon)

        driveSpeed = Speed

        steerAng = getSteering(currentHeading1, currentLat, currentLon, targetLat, targetLon)
        distFromTraget = getDistance(currentLat, currentLon, targetLat, targetLon)

        print('Steering Angle: ',steerAng)
        print('Distance from Target: ',distFromTraget)

        if distFromTraget > 3:
            drive = differentialDrive(steerAng, driveSpeed)
            LWSpeed = drive[0]
            RWSpeed = drive[1]
            print('Speed:\t',LWSpeed,'\t,',RWSpeed)
            robot.set_motors(LWSpeed,RWSpeed)

        else:
            print("Goal Reached")
            robot.set_motors(0,0)


## Defining Collision Avoidance
Inputs needed: Latitude and Longitude of the traget loctaion

In [ ]:
BOUNDARY_TH=0.2  # Area Threshold for boundary restriction
BOUNDARY_TH_SEN=0.05  # Area threshold for sensor
HEADING_SPEED=0.2

targLat = <Specify lat>
targLon = <Specify lat>

#Function for collision avoidance
def col_avoid(img,frame_id,lft,THRESH=250,row_start=470,DARK_TH=100):
    img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #ret,th=cv2.threshold(img_gray,5,255,cv2.THRESH_BINARY)
    img_gray[img_gray>THRESH]=255
    img_gray[img_gray!=255]=0
    th=img_gray
    cv2.imwrite('cam_out.jpg',th)
    
    CEN_TH=DARK_TH
    STBD_TH=DARK_TH
    PORT_TH=DARK_TH

    th_new=th[row_start::,100:500]
    th_port=th[row_start::,0:100]
    th_stbd=th[row_start::,500::]
    dark_area=th_new.shape[0]*th_new.shape[1]-cv2.countNonZero(th_new)
    port_area=th_port.shape[0]*th_port.shape[1]-cv2.countNonZero(th_port)
    stbd_area=th_stbd.shape[0]*th_stbd.shape[1]-cv2.countNonZero(th_stbd)
    print(f'port atea:{port_area},dark area:{dark_area},stbd area{stbd_area}')
    
# If no obstical is detected
# Then move towards the target location

    if dark_area<CEN_TH:
        goToTarget(HEADING_SPEED,targLat,targLon)
#         robot.forward(FORWARD_CMND)
        #robot.set_motors(0.3, 0.3-0.05)
        #time.sleep(0.4)
        #robot.stop()
        #print('forward path clear',dark_area)
        
# Heading Right or Left depending on the position of the obstical

    elif port_area<PORT_TH or stbd_area<STBD_TH:
        if port_area>stbd_area:
            robot.right(0.14)
            time.sleep(0.5)
            robot.stop()
        else:
            robot.left(0.15)
            time.sleep(0.5)
            robot.stop()
    else:
        #print('Dark zone.. moving back',dark_area)
        robot.backward(0.25)
        time.sleep(0.5)
        robot.stop()
        robot.left(0.15)
        time.sleep(0.5)
        robot.forward(0.15)
        time.sleep(0.5)
        robot.stop()
            # mov_circle_b(0.4,delay=1)
    #elif port_area<PORT_TH or stbd_area<STBD_TH:
   
    return lft

## Starting
Note: Robot will start moving

In [ ]:
lft=0
frame_id = 0
camera = Camera.instance(width=640, height=640)

try:
    while True:
        image = camera.value #change['new']
        frame_size = image.shape[:2]

        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            camera.stop()
        
        lft=col_avoid(image,frame_id,lft,THRESH=60,row_start=470)
        frame_id += 1
            
except KeyboardInterrupt:
    print('keyboard interrupt & stopping the camera')
    robot.stop()
    camera.stop()
    pass 